In [1]:
%pip install pandas requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import requests
import pandas as pd
from urllib.parse import urlparse
import time
import shutil
from datetime import datetime
import re
import pymorphy3
from nltk.corpus import stopwords
import nltk
import csv
from typing import List, Dict, Any

In [3]:
# Скачиваем стоп-слова если они еще не скачаны
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

In [4]:
def preprocess_text(text):
    """Предобработка текста: токенизация, лемматизация, удаление стоп-слов"""
    if pd.isna(text) or not isinstance(text, str):
        return []
    
    # Инициализируем лемматизатор
    morph = pymorphy3.MorphAnalyzer()
    
    # Получаем русские стоп-слова
    stop_words = set(stopwords.words('russian'))
    
    # Очищаем текст: оставляем только буквы и пробелы
    text_clean = re.sub(r'[^а-яёa-z\s]', ' ', text.lower(), flags=re.IGNORECASE)
    
    # Токенизируем
    tokens = text_clean.split()
    
    # Лемматизируем и фильтруем
    lemmatized_tokens = []
    for token in tokens:
        if len(token) > 2 and token not in stop_words:  # Игнорируем короткие слова и стоп-слова
            parsed = morph.parse(token)[0]
            lemma = parsed.normal_form
            lemmatized_tokens.append(lemma)
    
    return lemmatized_tokens

In [5]:
def find_ingredient_id_by_lemma(lemma: str, ingredients_file: str) -> int:
    """Находит ID ингредиента по лемме"""
    if not os.path.exists(ingredients_file):
        return None
    
    try:
        with open(ingredients_file, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter='\t')
            for row in reader:
                if row.get('lemma', '').lower() == lemma.lower():
                    return int(row['id'])
    except Exception as e:
        print(f"Ошибка при поиске ингредиента: {e}")
    
    return None

In [6]:
def find_ingredients_in_step(tokens_list):
    """
    Находит все ингредиенты в списке токенов
    """
    ingredients = set()  # используем set чтобы избежать дубликатов
    
    # Если tokens_list это строка, преобразуем в список
    if isinstance(tokens_list, str):
        try:
            tokens_list = ast.literal_eval(tokens_list)
        except:
            tokens_list = tokens_list.strip("[]").replace("'", "").split(", ")
    
    for token in tokens_list:
        ingredient = find_ingredient_id_by_lemma(token, "./data/ingredients.tsv")
        if ingredient:
            ingredients.add(ingredient)
    
    return list(ingredients) if ingredients else None

In [7]:
def process_images_and_update_tsv():
    # Пути к файлам
    raw_tsv_path = './data/steps_raw.tsv'
    processed_tsv_path = './data/steps.tsv'
    base_dir = './data/sources'
    
    # Создаем необходимые папки
    os.makedirs(base_dir, exist_ok=True)
    os.makedirs(os.path.dirname(processed_tsv_path), exist_ok=True)
    
    # Читаем исходный TSV файл
    try:
        df_raw = pd.read_csv(raw_tsv_path, sep='\t')
        print(f"Прочитано {len(df_raw)} строк из {raw_tsv_path}")
    except Exception as e:
        print(f"Ошибка чтения TSV файла: {e}")
        return
    
    # Создаем или читаем файл с обработанными данными
    if os.path.exists(processed_tsv_path):
        df_processed = pd.read_csv(processed_tsv_path, sep='\t')
        # Проверяем есть ли столбец tokens, если нет - добавляем
        if 'tokens' not in df_processed.columns:
            df_processed['tokens'] = [[] for _ in range(len(df_processed))]
    else:
        # Создаем копию структуры исходного файла без данных
        df_processed = df_raw.iloc[0:0].copy()
        # Добавляем столбец tokens
        df_processed['tokens'] = [[] for _ in range(len(df_processed))]
        # Заменяем столбец image_url на image
        if 'image_url' in df_processed.columns:
            df_processed = df_processed.rename(columns={'image_url': 'image'})
        elif 'url' in df_processed.columns:
            df_processed = df_processed.rename(columns={'url': 'image'})
    
    successful_downloads = 0
    failed_downloads = 0
    processed_rows = []
    delete_rows = []
    
    # Проходим по всем строкам исходного файла
    for index, row in df_raw.iterrows():
        image_url = str(row.get('image_url', '')).strip()
        
        # Пропускаем строки без URL изображения
        if not image_url or image_url == 'nan' or image_url == 'None':
            print(f"Пропуск строки {index}: нет URL изображения")
            continue
        
        recipe_id = str(row['recipe_id'])
        step_index = str(row['step_index'])
        
        # Создаем папку для recipe_id
        recipe_dir = os.path.join(base_dir, recipe_id)
        os.makedirs(recipe_dir, exist_ok=True)
        
        # Получаем расширение файла из URL
        parsed_url = urlparse(image_url)
        file_extension = os.path.splitext(parsed_url.path)[1]
        
        # Если расширение не найдено, используем .png по умолчанию
        if not file_extension:
            file_extension = '.png'
        
        # Формируем имя файла и путь
        filename = f"{step_index}{file_extension}"
        file_path = os.path.join(recipe_dir, filename)
        relative_image_path = f"./sources/{recipe_id}/{filename}"
        
        # Пропускаем если файл уже существует
        if os.path.exists(file_path):
            print(f"Файл уже существует: {file_path}")
            # Но все равно добавляем в обработанные
            delete_rows.append(index)
            successful_downloads += 1
            continue
        
        # Скачиваем изображение
        try:
            response = requests.get(image_url, stream=True, timeout=30)
            response.raise_for_status()
            
            with open(file_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        file.write(chunk)
            
            print(f"✓ Скачано: {file_path}")
            successful_downloads += 1
            processed_rows.append(index)
            
        except Exception as e:
            print(f"✗ Ошибка скачивания {image_url}: {e}")
            failed_downloads += 1
            continue
        
        # Небольшая пауза чтобы не перегружать сервер
        time.sleep(0.1)
    
    # Обновляем данные: переносим обработанные строки
    if processed_rows or delete_rows:
        # Копируем обработанные строки
        processed_data = df_raw.loc[processed_rows + delete_rows].copy()
        
        # Добавляем столбец tokens с токенизированным описанием
        processed_data['tokens'] = processed_data['description'].apply(preprocess_text)
        processed_data['ingredients'] = processed_data['tokens'].apply(find_ingredients_in_step) 
        # Заменяем столбец image_url на image с локальным путем
        if 'image_url' in processed_data.columns:
            for idx in processed_rows + delete_rows:
                recipe_id = str(df_raw.loc[idx, 'recipe_id'])
                step_index = str(df_raw.loc[idx, 'step_index'])
                file_extension = os.path.splitext(urlparse(df_raw.loc[idx, 'image_url']).path)[1] or '.png'
                processed_data.loc[idx, 'image'] = f"./sources/{recipe_id}/{step_index}{file_extension}"
            processed_data = processed_data.drop(columns=['image_url'])
        elif 'url' in processed_data.columns:
            for idx in processed_rows + delete_rows:
                recipe_id = str(df_raw.loc[idx, 'recipe_id'])
                step_index = str(df_raw.loc[idx, 'step_index'])
                file_extension = os.path.splitext(urlparse(df_raw.loc[idx, 'url']).path)[1] or '.png'
                processed_data.loc[idx, 'image'] = f"./sources/{recipe_id}/{step_index}{file_extension}"
            processed_data = processed_data.drop(columns=['url'])
        
        # Добавляем к обработанному файлу
        df_processed = pd.concat([df_processed, processed_data], ignore_index=True)
        
        # Удаляем обработанные строки из исходного файла
        df_remaining = df_raw.drop(processed_rows + delete_rows)
        
        # Сохраняем обновленные файлы
        if os.path.exists(processed_tsv_path):
            # Дописываем без заголовка
            processed_data.to_csv(processed_tsv_path, sep='\t', index=False, mode='a', header=False)
        else:
            # Создаем новый файл с заголовком
            processed_data.to_csv(processed_tsv_path, sep='\t', index=False)
        df_remaining.to_csv(raw_tsv_path, sep='\t', index=False)
        
        print(f"\nРезультат:")
        print(f"Успешно скачано: {successful_downloads}")
        print(f"Ошибок скачивания: {failed_downloads}")
        print(f"Перенесено строк: {len(processed_rows) + len(delete_rows)}")
        print(f"Осталось в raw: {len(df_remaining)}")
        print(f"Всего в processed: {len(df_processed)}")

    else:
        print("Не было обработано ни одной строки")

In [8]:
def create_backup_with_append(source_path, backup_path):
    """Создает бэкап с дописыванием содержимого"""
    if not os.path.exists(source_path):
        return
    
    # Читаем текущее содержимое исходного файла
    with open(source_path, 'r', encoding='utf-8') as source:
        content = source.read()
    
    # Добавляем timestamp и разделитель
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    backup_content = f"\n\n=== Backup {timestamp} ===\n{content}"
    
    # Дописываем в бэкап файл
    with open(backup_path, 'a', encoding='utf-8') as backup:
        backup.write(backup_content)
    
    print(f"Бэкап создан: {backup_path}")

In [9]:
def download_video(video_url, recipe_id, base_dir):
    """Скачивает видео и возвращает локальный путь"""
    if not video_url or pd.isna(video_url) or str(video_url).strip() in ['', 'nan', 'None']:
        return None
    
    video_url = str(video_url).strip()
    
    # Создаем папку для recipe_id
    recipe_dir = os.path.join(base_dir, str(recipe_id))
    os.makedirs(recipe_dir, exist_ok=True)
    
    # Получаем расширение файла из URL
    parsed_url = urlparse(video_url)
    file_extension = os.path.splitext(parsed_url.path)[1]
    
    # Если расширение не найдено, используем .mp4 по умолчанию
    if not file_extension:
        file_extension = '.mp4'
    
    # Формируем имя файла и путь
    filename = f"vid__{recipe_id}{file_extension}"
    file_path = os.path.join(recipe_dir, filename)
    relative_video_path = f"./sources/{recipe_id}/{filename}"
    
    # Пропускаем если файл уже существует
    if os.path.exists(file_path):
        print(f"Видео уже существует: {file_path}")
        return relative_video_path
    
    # Скачиваем видео
    try:
        print(f"Скачиваем видео: {video_url}")
        response = requests.get(video_url, stream=True, timeout=60)
        response.raise_for_status()
        
        # Получаем общий размер файла для прогресс-бара
        total_size = int(response.headers.get('content-length', 0))
        downloaded_size = 0
        
        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
                    downloaded_size += len(chunk)
                    # Выводим прогресс каждые 5MB
                    if total_size > 0 and downloaded_size % (5 * 1024 * 1024) == 0:
                        progress = (downloaded_size / total_size) * 100
                        print(f"Прогресс: {progress:.1f}% ({downloaded_size//1024//1024}MB/{total_size//1024//1024}MB)")
        
        print(f"✓ Видео скачано: {file_path}")
        return relative_video_path
        
    except Exception as e:
        print(f"✗ Ошибка скачивания видео {video_url}: {e}")
        # Удаляем частично скачанный файл
        if os.path.exists(file_path):
            os.remove(file_path)
        return None


In [10]:
def process_recipes_and_videos():
    """Обрабатывает рецепты и скачивает видео"""
    # Пути к файлам
    raw_tsv_path = './data/recipes_raw.tsv'
    processed_tsv_path = './data/recipes.tsv'
    base_dir = './data/sources'
    
    # Создаем необходимые папки
    os.makedirs(base_dir, exist_ok=True)
    os.makedirs(os.path.dirname(processed_tsv_path), exist_ok=True)
    
    # Читаем исходный TSV файл
    try:
        df_raw = pd.read_csv(raw_tsv_path, sep='\t')
        print(f"Прочитано {len(df_raw)} рецептов из {raw_tsv_path}")
    except Exception as e:
        print(f"Ошибка чтения TSV файла: {e}")
        return
    
    # Создаем или читаем файл с обработанными данными
    if os.path.exists(processed_tsv_path):
        df_processed = pd.read_csv(processed_tsv_path, sep='\t')
        # Проверяем есть ли столбец local_video_path, если нет - добавляем
        if 'local_video_path' not in df_processed.columns:
            df_processed['local_video_path'] = None
    else:
        # Создаем копию структуры исходного файла без данных
        df_processed = df_raw.iloc[0:0].copy()
        # Добавляем столбцы
        df_processed['local_video_path'] = None
    
    successful_downloads = 0
    failed_downloads = 0
    processed_rows = []
    
    # Проходим по всем строкам исходного файла
    for index, row in df_raw.iterrows():
        recipe_id = str(row['id'])
        video_url = str(row.get('mp4_url', '')).strip()
        youtube_url = str(row.get('youtube_url', '')).strip()
        title = str(row.get('title', ''))
        
        # Используем mp4_url, если он есть, иначе youtube_url
        download_url = video_url if video_url not in ['', 'nan', 'None'] else youtube_url
        
        # Пропускаем строки без URL видео
        if not download_url or download_url == 'nan' or download_url == 'None':
            print(f"Пропуск рецепта {recipe_id}: нет URL видео")
            continue
        
        # Скачиваем видео
        local_video_path = download_video(download_url, recipe_id, base_dir)
        
        if local_video_path:
            successful_downloads += 1
            processed_rows.append(index)
            print(f"✓ Успешно обработан рецепт {recipe_id}: {title}")
        else:
            failed_downloads += 1
            print(f"✗ Ошибка обработки рецепта {recipe_id}: {title}")
        
        # Небольшая пауза чтобы не перегружать сервер
        time.sleep(1)
    
    # Обновляем данные: переносим обработанные строки
    if processed_rows:
        # Копируем обработанные строки
        processed_data = df_raw.loc[processed_rows].copy()
        
        
        # Добавляем столбец с локальным путем к видео
        for idx in processed_rows:
            recipe_id = str(df_raw.loc[idx, 'id'])
            video_url = str(df_raw.loc[idx, 'mp4_url']).strip()
            youtube_url = str(df_raw.loc[idx, 'youtube_url']).strip()
            download_url = video_url if video_url not in ['', 'nan', 'None'] else youtube_url
            
            if download_url and download_url not in ['', 'nan', 'None']:
                parsed_url = urlparse(download_url)
                file_extension = os.path.splitext(parsed_url.path)[1] or '.mp4'
                processed_data.loc[idx, 'local_video_path'] = f"./sources/{recipe_id}/vid__{recipe_id}{file_extension}"
        
        # Добавляем к обработанному файлу (дописываем, а не перезаписываем)
        df_processed = pd.concat([df_processed, processed_data], ignore_index=True)
        
        # Удаляем обработанные строки из исходного файла
        df_remaining = df_raw.drop(processed_rows)
        
        # Сохраняем обновленные файлы
        if os.path.exists(processed_tsv_path):
            # Дописываем без заголовка
            processed_data.to_csv(processed_tsv_path, sep='\t', index=False, mode='a', header=False)
        else:
            # Создаем новый файл с заголовком
            processed_data.to_csv(processed_tsv_path, sep='\t', index=False)
        df_remaining.to_csv(raw_tsv_path, sep='\t', index=False)
        
        print(f"\nРезультат обработки рецептов:")
        print(f"Успешно скачано видео: {successful_downloads}")
        print(f"Ошибок скачивания: {failed_downloads}")
        print(f"Перенесено рецептов: {len(processed_rows)}")
        print(f"Осталось в raw: {len(df_remaining)}")
        print(f"Всего в processed: {len(df_processed)}")
        
        # Показываем пример токенизации
        if len(processed_data) > 0:
            print("\nПример токенизации названия:")
            sample_row = processed_data.iloc[0]
            print(f"Название: {sample_row['title']}")
    else:
        print("Не было обработано ни одного рецепта")

In [11]:
raw_tsv_path = './data/steps_raw.tsv'
backup_path = './data/steps_raw_backup.tsv'
    
create_backup_with_append(raw_tsv_path, backup_path)
    
    # Запускаем обработку
process_images_and_update_tsv()

Бэкап создан: ./data/steps_raw_backup.tsv
Прочитано 0 строк из ./data/steps_raw.tsv
Не было обработано ни одной строки


In [12]:
raw_tsv_path = './data/recipes_raw.tsv'
backup_path = './data/recipes_raw_backup.tsv'
    
create_backup_with_append(raw_tsv_path, backup_path)
    
    # Запускаем обработку
process_recipes_and_videos()

Бэкап создан: ./data/recipes_raw_backup.tsv
Прочитано 0 рецептов из ./data/recipes_raw.tsv
Не было обработано ни одного рецепта


In [ ]:
import pandas as pd
import os
import subprocess
import re

def time_to_seconds(time_str):
    """Конвертирует время из формата hh:mm:ss или mm:ss в секунды"""
    parts = list(map(int, time_str.split(':')))
    
    if len(parts) == 3:  # hh:mm:ss
        hours, minutes, seconds = parts
        return hours * 3600 + minutes * 60 + seconds
    elif len(parts) == 2:  # mm:ss
        minutes, seconds = parts
        return minutes * 60 + seconds
    elif len(parts) == 1:  # ss
        return parts[0]
    else:
        raise ValueError(f"Неверный формат времени: {time_str}")

def seconds_to_time(seconds):
    """Конвертирует секунды в формат hh:mm:ss"""
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

def get_time_input(prompt):
    """Получает корректный ввод времени от пользователя"""
    while True:
        try:
            time_str = input(prompt).strip()
            if not time_str:
                return None
            return time_to_seconds(time_str)
        except ValueError:
            print("Неверный формат времени. Используйте hh:mm:ss, mm:ss или ss")


: 

In [12]:
def proceess_video():
    # Загружаем данные
    steps_df = pd.read_csv('./data/steps.tsv', sep='\t')
    recipes_df = pd.read_csv('./data/recipes.tsv', sep='\t')
    
    # Проверяем или создаем файл timecode.tsv
    timecode_file = './data/timecode.tsv'
    if os.path.exists(timecode_file):
        timecode_df = pd.read_csv(timecode_file, sep='\t')
        processed_step_ids = set(timecode_df['step_id'])
    else:
        timecode_df = pd.DataFrame(columns=['step_id', 'start_time', 'end_time', 'start_time_str', 'end_time_str'])
        processed_step_ids = set()
    
    # Находим шаги, которые еще не обработаны
    unprocessed_steps = steps_df[~steps_df['id'].isin(processed_step_ids)].copy()
    
    if unprocessed_steps.empty:
        print("Все шаги уже обработаны!")
        return
    
    print(f"Найдено {len(unprocessed_steps)} необработанных шагов")
    
    # Сортируем по recipe_id и step_number для удобства
    unprocessed_steps = unprocessed_steps.sort_values(['step_number']).sort_values(['recipe_id'])
    
    # Создаем словарь с информацией о видео для каждого рецепта
    video_paths = {}
    for _, recipe in recipes_df.iterrows():
        video_paths[recipe['id']] = recipe['local_video_path']
    hist_recipe_id=None
    hist_time_end=None
    hist_step=None
    # Обрабатываем каждый необработанный шаг
    for _, step in unprocessed_steps.sort_values(['id']).iterrows():
        step_id = step['id']
        recipe_id = step['recipe_id']
        step_number = step['step_number']
        description = step['description']
        print(f"\n{'='*80}")
        print(f"Рецепт ID: {recipe_id}, Шаг {step_number}: {description}")
        print(f"{'='*80}")
        
        # Проверяем наличие видео
        video_path = "./data"+re.sub(r'^\.', '', video_paths.get(recipe_id))
        
        # Запрашиваем временные метки
        print("\nВведите временные метки для этого шага:")
        print("Формат: hh:mm:ss, mm:ss или ss (оставьте пустым для пропуска)")
        if hist_time_end and hist_recipe_id==recipe_id and hist_step+1==int(step['step_index']):
            start_seconds=hist_time_end
        else:
            start_seconds = get_time_input(f"Начало отрезка (для шага {step_number} {description}, видео {video_path}): ")
        if start_seconds is None:
            print("Пропускаем шаг...")
            continue
            
        end_seconds = get_time_input(f"Конец отрезка (для шага {step_number} {description}, видео {video_path}): ")
        hist_time_end=end_seconds
        if end_seconds is None:
            print("Пропускаем шаг...")
            continue
        
        # Проверяем корректность временного интервала
        if end_seconds <= start_seconds:
            print("Ошибка: конечное время должно быть больше начального!")
            continue
        hist_recipe_id=recipe_id
        hist_step=int(step['step_index'])
        # Добавляем запись в timecode_df
        new_row = {
            'step_id': step_id,
            'start_time': start_seconds,
            'end_time': end_seconds,
            'start_time_str': seconds_to_time(start_seconds),
            'end_time_str': seconds_to_time(end_seconds)
        }
        
        timecode_df = pd.concat([timecode_df, pd.DataFrame([new_row])], ignore_index=True)
        
        # Сохраняем после каждого шага
        timecode_df.to_csv(timecode_file, sep='\t', index=False)
        print(f"Сохранено для шага {step_id}")
    
    print("\nОбработка завершена!")
proceess_video()

Найдено 20 необработанных шагов

Рецепт ID: 35, Шаг Шаг 2:: Нарежьте мясо ка куски среднего размера.

Введите временные метки для этого шага:
Формат: hh:mm:ss, mm:ss или ss (оставьте пустым для пропуска)
Пропускаем шаг...

Рецепт ID: 35, Шаг Шаг 12:: Нарежьте капусту на квадратные кусочки, не мелко.  Добавьте капусту поверх слоя картофеля, чеснока и специй. Посолите сверху.

Введите временные метки для этого шага:
Формат: hh:mm:ss, mm:ss или ss (оставьте пустым для пропуска)
Сохранено для шага 510

Рецепт ID: 35, Шаг Шаг 13:: Накройте кастрюлю крышкой и тушите 40 минут на огне ниже среднего.

Введите временные метки для этого шага:
Формат: hh:mm:ss, mm:ss или ss (оставьте пустым для пропуска)
Сохранено для шага 511

Рецепт ID: 35, Шаг Шаг 14:: Перемешайте. Мясо не следует обжаривать слишком долго, иначе можно его пересушить, достаточно добиться легкой корочки. Овощи для дымлямы должны быть нарезаны крупно, чтобы они не превратились к концу готовки в неаппетитное месиво. Овощи следует в